In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
#from torch.optim import Adam
#import torch.nn.functional as F
import math

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

class TimeSeriesDataset(Dataset):
    def __init__(self, dataframe):
        self.X = torch.tensor(dataframe.iloc[:, :3].values, dtype=torch.float32)  # Input: [Batch, 3]
        self.Y = torch.tensor(dataframe.iloc[:, 3:].values, dtype=torch.float32).unsqueeze(-1) # Output: [Batch, 255, 1]

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len):
        super(PositionalEncoding, self).__init__()
        self.encoding = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * -(math.log(10000.0) / d_model))
        self.encoding[:, 0::2] = torch.sin(position * div_term)
        self.encoding[:, 1::2] = torch.cos(position * div_term)
        self.encoding = self.encoding.unsqueeze(0)  # Add batch dimension

    def forward(self, x):
        return x + self.encoding[:, :x.size(1), :].to(x.device)

# Transformer Model
class TransformerTimeSeriesModel(nn.Module):
    def __init__(self, input_dim, output_dim, seq_length, d_model, nhead, num_layers, dim_feedforward):
        super(TransformerTimeSeriesModel, self).__init__()
        self.input_dim = input_dim
        self.d_model = d_model
        self.seq_length = seq_length
 
        # Input Encoder (maps input to d_model size)
        self.encoder = nn.Linear(input_dim, d_model)  # (Batch, 3) -> (Batch, d_model)
        
        # Project input to match the sequence length
        self.expand_input = nn.Linear(d_model, seq_length * d_model)  # (Batch, d_model) -> (Batch, seq_length * d_model)
        
        # Target embedding for decoder input
        self.target_embedding = nn.Linear(1, d_model)  # New embedding layer for target sequence
  
        # Positional Encoding for Time Steps
        self.pos_encoder = PositionalEncoding(d_model, seq_length)
        
        # Transformer Decoder
        decoder_layer = nn.TransformerDecoderLayer(d_model=d_model, nhead=nhead, dim_feedforward=dim_feedforward)
        self.transformer_decoder = nn.TransformerDecoder(decoder_layer, num_layers=num_layers)
        
        # Final Output Layer
        self.output_layer = nn.Linear(d_model, output_dim)  # (Batch, 255, d_model) -> (Batch, 255, 1)

    def forward(self, x, target_seq):
        # x: Input features [Batch, 3]
        # target_seq: Target sequence for teacher forcing [Batch, 255, 1]
        
        # Encode input features
        encoded_input = self.encoder(x)  # [Batch, d_model]
        
        # Expand input to match sequence length
        expanded_input = self.expand_input(encoded_input)  # [Batch, seq_length * d_model]
        expanded_input = expanded_input.view(-1, self.seq_length, self.d_model)  # Reshape to [Batch, 255, d_model]
        
        # Add Positional Encoding
        expanded_input = self.pos_encoder(expanded_input)
        
        # Process the target sequence through the same encoding pipeline
  #      target_embeddings = self.encoder(target_seq)
  #      target_embeddings = nn.Linear(1, d_model)(target_seq)  # [Batch, 255, d_model]
        target_embeddings = self.target_embedding(target_seq)  # [Batch, 255, d_model]
        target_embeddings = self.pos_encoder(target_embeddings)
        
        # Decode sequence
        output = self.transformer_decoder(
            tgt=target_embeddings, memory=expanded_input
        )  # Output shape: [Batch, 255, d_model]
        
        # Map to output dimensions
        predictions = self.output_layer(output)  # [Batch, 255, 1]
        return predictions
    
def train_model(model, dataloader, optimizer, loss_fn, num_epochs, device):
    loss_list = list()
    model.to(device)
    for epoch in range(num_epochs):
        model.train()
        for batch in dataloader:
            x, y = batch  # x: [Batch, N], y: [Batch, T]
            x, y = x.to(device), y.to(device)
            
            # Prepare target for teacher forcing
            target_seq = y 
            #target_seq = y[:, :-1]  # All except last time step
            #actual = y[:, 1:]       # All except first time step
            
            # Forward pass
            output = model(x, target_seq)
            loss = loss_fn(output, y)
            
            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        loss_list.append(loss.item())
        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}")
    return loss_list


In [5]:
##################################################################
# Set up      
##################################################################

print("Importing data...\n")

data_output = pd.read_csv("~/Desktop/TS-Clustering/SimData/epsteinCV_outputs_active.csv", header=None)
print("Max value is :", data_output.to_numpy().max(), "\n")

scaler = MinMaxScaler(feature_range=(0, data_output.to_numpy().max()))
scaler.fit(data_output)
data_output = scaler.transform(data_output)
data_output = pd.DataFrame(data_output)

data_input = pd.read_csv("~/Desktop/TS-Clustering/SimData/epsteinCV_inputs.csv", sep=" ", header=None)
data = pd.concat([data_input, data_output], axis=1)

# Split the data into training and validation sets
train_data, valid_data = train_test_split(data, test_size=0.2, random_state=42)

# Save the validation set to a new CSV file
valid_data.to_csv("validation_set_epstein.csv", index=False)


Importing data...

Max value is : 1451 



In [6]:
##################################################################
# Model 1      
##################################################################

print("Starting model 1\n")

dataset = TimeSeriesDataset(train_data)

batch_size = 32
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

input_dim = 3      # Number of input features
output_dim = 1     # Predicting one value per time step
seq_length = 252   # Number of time steps in output
d_model = 128      # Embedding dimension for Transformer
nhead = 4          # Number of attention heads
num_layers = 2     # Number of Transformer layers
dim_feedforward = 512  # Feedforward network size

# Instantiate the model

print("Instatiate \n")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TransformerTimeSeriesModel(
    input_dim, output_dim, seq_length, d_model, nhead, num_layers, dim_feedforward
).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = torch.nn.MSELoss()  # Regression loss

print("Train\n")
# Training loop
num_epochs = 20  # Adjust based on dataset size and performance
loss = train_model(model, dataloader, optimizer, loss_fn, num_epochs, device)

df = pd.DataFrame(loss, columns=["loss"])
df.to_csv('transformer_adam_lr001_epstein_loss.csv', index=False)
torch.save(model.state_dict(), "transformer_adam_lr001_epstein.pth")


Starting model 1

Instatiate 

Train

Epoch 1/20, Loss: 331657.125
Epoch 2/20, Loss: 553.4930419921875
Epoch 3/20, Loss: 23349.3515625
Epoch 4/20, Loss: 528554.1875
Epoch 5/20, Loss: 36697.60546875
Epoch 6/20, Loss: 17380.27734375
Epoch 7/20, Loss: 20159.275390625
Epoch 8/20, Loss: 223107.234375
Epoch 9/20, Loss: 370103.96875
Epoch 10/20, Loss: 883563.0
Epoch 11/20, Loss: 483048.4375
Epoch 12/20, Loss: 7868.3876953125
Epoch 13/20, Loss: 4206.35595703125
Epoch 14/20, Loss: 2425.756103515625
Epoch 15/20, Loss: 25472.064453125
Epoch 16/20, Loss: 1116486.25
Epoch 17/20, Loss: 11565.4912109375
Epoch 18/20, Loss: 849.1578369140625
Epoch 19/20, Loss: 14909.6806640625
Epoch 20/20, Loss: 29997.72265625


In [7]:
##################################################################
# Model 2      
##################################################################

print("Starting model 2\n")

dataset = TimeSeriesDataset(train_data)

batch_size = 32
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

input_dim = 3      # Number of input features
output_dim = 1     # Predicting one value per time step
seq_length = 252   # Number of time steps in output
d_model = 128      # Embedding dimension for Transformer
nhead = 4          # Number of attention heads
num_layers = 2     # Number of Transformer layers
dim_feedforward = 512  # Feedforward network size

# Instantiate the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TransformerTimeSeriesModel(
    input_dim, output_dim, seq_length, d_model, nhead, num_layers, dim_feedforward
).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn = torch.nn.MSELoss()  # Regression loss

# Training loop
num_epochs = 20  # Adjust based on dataset size and performance
loss = train_model(model, dataloader, optimizer, loss_fn, num_epochs, device)

df = pd.DataFrame(loss, columns=["loss"])
df.to_csv('transformer_adam_lr01_epstein_loss.csv', index=False)
torch.save(model.state_dict(), "transformer_adam_lr01_epstein.pth")



Starting model 2

Epoch 1/20, Loss: 24429.005859375
Epoch 2/20, Loss: 19443.615234375
Epoch 3/20, Loss: 16719.541015625
Epoch 4/20, Loss: 732787.0625
Epoch 5/20, Loss: 18382.765625
Epoch 6/20, Loss: 20425.94921875
Epoch 7/20, Loss: 17974.16796875
Epoch 8/20, Loss: 24082.98046875
Epoch 9/20, Loss: 18664.146484375
Epoch 10/20, Loss: 19070.171875
Epoch 11/20, Loss: 16628.61328125
Epoch 12/20, Loss: 18840.94921875
Epoch 13/20, Loss: 18188.232421875
Epoch 14/20, Loss: 21944.369140625
Epoch 15/20, Loss: 21960.646484375
Epoch 16/20, Loss: 20113.146484375
Epoch 17/20, Loss: 19022.21484375
Epoch 18/20, Loss: 23376.65625
Epoch 19/20, Loss: 12144.6728515625
Epoch 20/20, Loss: 14653.7939453125


In [ ]:
##################################################################
# Model 3      
##################################################################

print("Starting model 3\n")

dataset = TimeSeriesDataset(train_data)

batch_size = 32
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

input_dim = 3      # Number of input features
output_dim = 1     # Predicting one value per time step
seq_length = 252   # Number of time steps in output
d_model = 128      # Embedding dimension for Transformer
nhead = 4          # Number of attention heads
num_layers = 2     # Number of Transformer layers
dim_feedforward = 512  # Feedforward network size

# Instantiate the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TransformerTimeSeriesModel(
    input_dim, output_dim, seq_length, d_model, nhead, num_layers, dim_feedforward
).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
loss_fn = torch.nn.MSELoss()  # Regression loss

# Training loop
num_epochs = 20  # Adjust based on dataset size and performance
loss = train_model(model, dataloader, optimizer, loss_fn, num_epochs, device)

df = pd.DataFrame(loss, columns=["loss"])
df.to_csv('transformer_adam_lr0001_epstein_loss.csv', index=False)
torch.save(model.state_dict(), "transformer_adam_lr0001_epstein.pth")


In [ ]:
##################################################################
# Model 4      
##################################################################

print("Starting model 4\n")

dataset = TimeSeriesDataset(train_data)

batch_size = 32
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

input_dim = 3      # Number of input features
output_dim = 1     # Predicting one value per time step
seq_length = 252   # Number of time steps in output
d_model = 128      # Embedding dimension for Transformer
nhead = 4          # Number of attention heads
num_layers = 2     # Number of Transformer layers
dim_feedforward = 512  # Feedforward network size

# Instantiate the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TransformerTimeSeriesModel(
    input_dim, output_dim, seq_length, d_model, nhead, num_layers, dim_feedforward
).to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)
loss_fn = torch.nn.MSELoss()  # Regression loss

# Training loop
num_epochs = 20  # Adjust based on dataset size and performance
loss = train_model(model, dataloader, optimizer, loss_fn, num_epochs, device)

df = pd.DataFrame(loss, columns=["loss"])
df.to_csv('transformer_adamW_lr001_epstein_loss.csv', index=False)
torch.save(model.state_dict(), "transformer_adamW_lr001_epstein.pth")



In [ ]:
##################################################################
# Model 5      
##################################################################

print("Starting model 5\n")

dataset = TimeSeriesDataset(train_data)

batch_size = 32
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

input_dim = 3      # Number of input features
output_dim = 1     # Predicting one value per time step
seq_length = 252   # Number of time steps in output
d_model = 128      # Embedding dimension for Transformer
nhead = 4          # Number of attention heads
num_layers = 2     # Number of Transformer layers
dim_feedforward = 512  # Feedforward network size

# Instantiate the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TransformerTimeSeriesModel(
    input_dim, output_dim, seq_length, d_model, nhead, num_layers, dim_feedforward
).to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=0.01)
loss_fn = torch.nn.MSELoss()  # Regression loss

# Training loop
num_epochs = 20  # Adjust based on dataset size and performance
loss = train_model(model, dataloader, optimizer, loss_fn, num_epochs, device)

df = pd.DataFrame(loss, columns=["loss"])
df.to_csv('transformer_adamW_lr01_epstein_loss.csv', index=False)
torch.save(model.state_dict(), "transformer_adamW_lr01_epstein.pth")



In [ ]:
##################################################################
# Model 6      
##################################################################

print("Starting model 6\n")

dataset = TimeSeriesDataset(train_data)

batch_size = 32
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

input_dim = 3      # Number of input features
output_dim = 1     # Predicting one value per time step
seq_length = 252   # Number of time steps in output
d_model = 128      # Embedding dimension for Transformer
nhead = 4          # Number of attention heads
num_layers = 2     # Number of Transformer layers
dim_feedforward = 512  # Feedforward network size

# Instantiate the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TransformerTimeSeriesModel(
    input_dim, output_dim, seq_length, d_model, nhead, num_layers, dim_feedforward
).to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=0.0001)
loss_fn = torch.nn.MSELoss()  # Regression loss

# Training loop
num_epochs = 20  # Adjust based on dataset size and performance
loss = train_model(model, dataloader, optimizer, loss_fn, num_epochs, device)

df = pd.DataFrame(loss, columns=["loss"])
df.to_csv('transformer_adamW_lr0001_epstein_loss.csv', index=False)
torch.save(model.state_dict(), "transformer_adamW_lr0001_epstein.pth")
